## Setup Conda Environment & Activate It

In [ ]:
# install miniconda
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh -b

# create environment
conda env create -f environment.yml
conda activate HunyuanVideo
conda install gcc_linux-64 gxx_linux-64 -y conda install cuda -c nvidia -y

## Install Dependencies

In [ ]:
pip install -r requirements.txt
pip install packaging
pip install ninja
pip install git+https://github.com/Dao-AILab/flash-attention.git@v2.5.9.post1

# hf_transfer = for hugging-face high speed transfers
pip install huggingface_hub[hf_transfer]


## Download Models & Text Encoders from HF

In [ ]:
read -sp "Enter or paste your Hugging Face access token: " HF_TOKEN
echo

huggingface-cli login --token $HF_TOKEN
export HF_HUB_ENABLE_HF_TRANSFER=1

# change dir to ckpts
cd ckpts

# download the pretrained models
huggingface-cli download tencent/HunyuanVideo --local-dir .

# Download Text Encoder
huggingface-cli download xtuner/llava-llama-3-8b-v1_1-transformers --local-dir ./llava-llama-3-8b-v1_1-transformers

# separate the lang model parts of llava-llama-3-8b-v1_1-transformers into text_encoder to conserve vram
python3 ../hyvideo/utils/preprocess_text_encoder_tokenizer_utils.py --input_dir llava-llama-3-8b-v1_1-transformers --output_dir text_encoder

# download clip text encoder
huggingface-cli download openai/clip-vit-large-patch14 --local-dir ./text_encoder_2

# go back one dir
cd ..

## Start Inferencing

In [ ]:
python3 sample_video.py \
--video-size 720 1280 \
--video-length 129 \
--infer-steps 30 \
--prompt " a teddy bear dancing, realistic." \
--flow-reverse \
--seed 0 \
--use-cpu-offload \
--save-path ./results\